In [ ]:
import joblib
import pandas as pd
import numpy as np
from typing import Dict, List

# --- 1. 저장된 모델 및 임계값 로드 (경로를 'models/'로 가정) ---
# NOTE: 실제 파일 경로에 따라 'models/' 부분을 수정해야 합니다.
try:
    LOADED_MODEL = joblib.load('models/final_churn_model.pkl')
    LOADED_THRESHOLDS = joblib.load('models/risk_thresholds.pkl')
    # 학습된 모델의 피처 목록 (컬럼 이름과 순서)
    # XGBoost/GradientBoostingClassifier는 'feature_names_in_' 속성을 가집니다.
    MODEL_FEATURES: List[str] = list(LOADED_MODEL.feature_names_in_) 
    
except FileNotFoundError:
    print("오류: 모델 파일(final_churn_model.pkl 또는 risk_thresholds.pkl)을 'models/' 경로에서 찾을 수 없습니다.")
    print("파일 경로를 확인하거나, 'model_pipeline.py'를 먼저 실행하여 파일을 생성해야 합니다.")
    raise

# --- 2. 전처리 함수 수정 및 완성 ---

def preprocess_data(df_input: pd.DataFrame) -> pd.DataFrame:
    """새로운 입력 데이터를 모델이 요구하는 형태로 전처리합니다."""
    
    # 입력 데이터의 복사본을 만들어 원본 데이터 변경을 방지
    df_temp = df_input.copy()

    # 1. Feature Engineering (파생변수 생성)
    df_temp['recent_3m_spent'] = df_temp['spent_m1'] + df_temp['spent_m2'] + df_temp['spent_m3']
    df_temp['past_3m_spent'] = df_temp['spent_m4'] + df_temp['spent_m5'] + df_temp['spent_m6']
    # 분모에 1을 더하여 0 나누기 오류 방지
    df_temp['spent_change_ratio'] = df_temp['recent_3m_spent'] / (df_temp['past_3m_spent'] + 1)
    
    spent_cols_m1_m6 = [f'spent_m{i}' for i in range(1, 7)]
    def calculate_inactivity(row):
        months_inactive = 0
        for month_spent in row[spent_cols_m1_m6]:
            if month_spent < 10000: months_inactive += 1
            else: break
        return months_inactive
    df_temp['inactivity_months'] = df_temp.apply(calculate_inactivity, axis=1) # <-- 누락된 파생변수 추가

    # 2. Feature Filtering (학습 시 사용된 최종 핵심 변수 목록 재정의)
    # 범주형 변수를 포함하여 One-Hot Encoding 대상 지정
    CORE_CATEGORICAL_FEATURES = ['gender', 'region', 'income_band', 'card_grade']
    
    # 학습 시 사용된 모든 수치형/파생 변수 (실제 모델 학습 시 컬럼과 일치해야 함)
    # 이 목록은 실제 final_churn_model.pkl을 만들 때 사용한 최종 목록이어야 합니다.
    CORE_NUMERIC_FEATURES = ['age', 'tenure_months', 'complaints_6m', 'marketing_open_rate_6m',
                             'spent_m1', 'txn_m1', 'login_m1', 
                             'spent_change_ratio', 'inactivity_months', # <--- 핵심 변수
                             'customer_id' # ID는 One-Hot Encoding에서 제외
                            ]
    
    # 사용할 모든 컬럼의 목록
    ALL_CORE_FEATURES = CORE_NUMERIC_FEATURES + CORE_CATEGORICAL_FEATURES
    
    # 사용할 컬럼만 선택
    df_filtered = df_temp[ALL_CORE_FEATURES].copy() 
    
    # 3. One-Hot Encoding (범주형 변수에 대해만 수행)
    df_encoded = pd.get_dummies(df_filtered, columns=CORE_CATEGORICAL_FEATURES, dtype=int)
    
    # 4. 컬럼 순서 정렬 및 누락 컬럼 추가 (가장 중요!)
    # 학습 데이터에 없었지만 예측 데이터에 새로 생길 수 있는 One-Hot 컬럼은 0으로 채워짐
    df_processed = df_encoded.reindex(columns=MODEL_FEATURES, fill_value=0)
    
    # customer_id는 예측에 사용하지 않으므로 제거 후 반환
    if 'customer_id' in df_processed.columns:
        return df_processed.drop(columns=['customer_id'])
    
    return df_processed

# --- 3. 예측 및 티어 분류 함수 ---

def predict_and_classify(df_new_data: pd.DataFrame) -> pd.DataFrame:
    """새로운 데이터를 받아 확률을 예측하고 위험 티어를 분류합니다."""
    
    # 1. 전처리 및 예측
    df_processed = preprocess_data(df_new_data.copy())
    churn_proba = LOADED_MODEL.predict_proba(df_processed)[:, 1]
    
    df_output = df_new_data[['customer_id']].copy() # customer_id만 추출
    df_output['churn_proba'] = churn_proba

    # 2. 티어 분류 (저장된 임계값 사용)
    T99 = LOADED_THRESHOLDS['T99']
    T95 = LOADED_THRESHOLDS['T95']
    T90 = LOADED_THRESHOLDS['T90']
    
    def assign_risk_tier(proba):
        if proba >= T99:
            return 'Tier 1' # Extreme Risk
        elif proba >= T95:
            return 'Tier 2' # Very High Risk
        elif proba >= T90:
            return 'Tier 3' # High Risk
        else:
            return 'Tier 4' # Low Risk

    df_output['risk_tier'] = df_output['churn_proba'].apply(assign_risk_tier)
    
    return df_output[['customer_id', 'churn_proba', 'risk_tier']]